<a href="https://colab.research.google.com/github/kikiru328/enterprise_analysis/blob/main/New_clf_ratio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')
my_path = '/content/notebooks'
os.symlink('/content/drive/My Drive/Colab Notebooks', my_path)
sys.path.insert(0,my_path)

Mounted at /content/drive


In [2]:
#  Directory 변경 (모듈이용)
%pwd
%cd '../content/notebooks'

# module import
## Data Load
from pickle5 import pickle 

## Basic module
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')


/content/drive/My Drive/Colab Notebooks


In [3]:
with open('/content/drive/MyDrive/공유문서/1차 프로젝트/5조/훈팡/새분류.pkl','rb') as file:
  df = pickle.load(file)


In [4]:
df.새분류.unique()

array(['비내구재', '내구재', '준내구재', '사치품'], dtype=object)

In [5]:
df_1 = df[(df['구매년도']==2014)&(df['구매월']>=1) & (df['구매월']<=6)] 
df_2 = df[(df['구매년도']==2014)&(df['구매월']>=7) & (df['구매월']<=12)]
df_3 = df[(df['구매년도']==2015)&(df['구매월']>=1) & (df['구매월']<=6)]
df_4 = df[(df['구매년도']==2015)&(df['구매월']>=7) & (df['구매월']<=12)]

In [6]:
t = pd.get_dummies(df_1['새분류'])
print(t.shape)
print(df_1.shape)
t

(6704092, 4)
(6704092, 17)


,내구재,비내구재,사치품,준내구재
52,1,0,0,0
53,1,0,0,0
54,0,0,0,1
55,0,1,0,0
78,0,1,0,0
...,...,...,...,...
12129306,0,1,0,0
12129307,0,1,0,0
12129367,0,1,0,0
12129368,0,1,0,0


In [10]:
def ratio(df):
  test = df[['고객번호','새분류']]
  test = test.set_index('고객번호')
  A = pd.get_dummies(test['새분류']).reset_index()
  A = A.groupby('고객번호').sum().astype(int)
  Li = list(A.columns)
  A['total'] = A.sum(axis=1).astype(int)
  for i in Li:
    A[f"{i}_r"] = A[i] / A['total']
  B = A.drop(['내구재','사치품','비내구재','준내구재','total'],axis=1)
  B['total_r'] = B.sum(axis=1)
  return B

In [11]:
H1_r = ratio(df_1)
H2_r = ratio(df_2)
H3_r = ratio(df_3)
H4_r = ratio(df_4)

In [12]:
H1_r

,내구재_r,비내구재_r,사치품_r,준내구재_r,total_r
고객번호,,,,,
1,0.004202,0.693277,0.000000,0.302521,1.0
2,0.032448,0.823009,0.000000,0.144543,1.0
3,0.000000,0.997126,0.000000,0.002874,1.0
4,0.000000,0.851852,0.000000,0.148148,1.0
5,0.002653,0.952255,0.002653,0.042440,1.0
...,...,...,...,...,...
19356,0.000000,1.000000,0.000000,0.000000,1.0
19357,0.000000,0.750000,0.000000,0.250000,1.0
19359,0.000000,1.000000,0.000000,0.000000,1.0


In [13]:
def reName(a,df):
    cols = list(df.columns)
    for i in cols:
      df[f'{a}_{i}'] = df[i]
    A = df.drop(cols,axis=1)
    return A

H1 = reName('H1',H1_r)
H2 = reName('H2',H2_r)
H3 = reName('H3',H3_r)
H4 = reName('H4',H4_r)

In [14]:
H1

,H1_내구재_r,H1_비내구재_r,H1_사치품_r,H1_준내구재_r,H1_total_r
고객번호,,,,,
1,0.004202,0.693277,0.000000,0.302521,1.0
2,0.032448,0.823009,0.000000,0.144543,1.0
3,0.000000,0.997126,0.000000,0.002874,1.0
4,0.000000,0.851852,0.000000,0.148148,1.0
5,0.002653,0.952255,0.002653,0.042440,1.0
...,...,...,...,...,...
19356,0.000000,1.000000,0.000000,0.000000,1.0
19357,0.000000,0.750000,0.000000,0.250000,1.0
19359,0.000000,1.000000,0.000000,0.000000,1.0


In [15]:
def new_train_df(a,b):
    A = pd.concat([a,b],axis=1)
    return A

train = new_train_df(H1,H2)

In [16]:
train

,H1_내구재_r,H1_비내구재_r,H1_사치품_r,H1_준내구재_r,H1_total_r,H2_내구재_r,H2_비내구재_r,H2_사치품_r,H2_준내구재_r,H2_total_r
고객번호,,,,,,,,,,
1,0.004202,0.693277,0.000000,0.302521,1.0,0.061798,0.716292,0.000000,0.221910,1.0
2,0.032448,0.823009,0.000000,0.144543,1.0,0.011070,0.815498,0.003690,0.169742,1.0
3,0.000000,0.997126,0.000000,0.002874,1.0,0.000000,0.997442,0.000000,0.002558,1.0
4,0.000000,0.851852,0.000000,0.148148,1.0,0.023256,0.761628,0.000000,0.215116,1.0
5,0.002653,0.952255,0.002653,0.042440,1.0,0.015385,0.946154,0.000000,0.038462,1.0
...,...,...,...,...,...,...,...,...,...,...
19356,0.000000,1.000000,0.000000,0.000000,1.0,0.002625,0.997375,0.000000,0.000000,1.0
19357,0.000000,0.750000,0.000000,0.250000,1.0,0.000000,0.941176,0.002353,0.056471,1.0
19359,0.000000,1.000000,0.000000,0.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.0


In [18]:
# trainlist = ['H1','H2','H3']
# c_li = list(H1_r.columns)


# trainList = []
# for col in c_li:
#   for h in trainlist:
#     trainList.append(f'{h}_{col}')

# train = train[trainList]

train = train[['H1_내구재_r',
 'H2_내구재_r',
 'H1_준내구재_r',
 'H2_준내구재_r',
  'H1_비내구재_r',
 'H2_비내구재_r',
 'H1_사치품_r',
 'H2_사치품_r',
]]
# trainList

In [19]:
train = train.reset_index()
train

,고객번호,H1_내구재_r,H2_내구재_r,H1_준내구재_r,H2_준내구재_r,H1_비내구재_r,H2_비내구재_r,H1_사치품_r,H2_사치품_r
0,1,0.004202,0.061798,0.302521,0.221910,0.693277,0.716292,0.000000,0.000000
1,2,0.032448,0.011070,0.144543,0.169742,0.823009,0.815498,0.000000,0.003690
2,3,0.000000,0.000000,0.002874,0.002558,0.997126,0.997442,0.000000,0.000000
3,4,0.000000,0.023256,0.148148,0.215116,0.851852,0.761628,0.000000,0.000000
4,5,0.002653,0.015385,0.042440,0.038462,0.952255,0.946154,0.002653,0.000000
...,...,...,...,...,...,...,...,...,...
19080,19356,0.000000,0.002625,0.000000,0.000000,1.000000,0.997375,0.000000,0.000000
19081,19357,0.000000,0.000000,0.250000,0.056471,0.750000,0.941176,0.000000,0.002353
19082,19359,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000
19083,19360,0.000000,0.000000,0.000000,0.011070,1.000000,0.988930,0.000000,0.000000


In [20]:
with open('/content/drive/MyDrive/공유문서/1차 프로젝트/5조/1012발표도못하고수정/10월14일이후 새로추가/train/new_ratio.pkl','wb') as train_save:
  pickle.dump(train,train_save)

In [21]:
def new_test_df(a,b,c):
    A = pd.concat([a,b],axis=1)
    train = pd.concat([A,c],axis=1)    
    return train

test = new_test_df(H1,H2,H3)

In [22]:
test

,H1_내구재_r,H1_비내구재_r,H1_사치품_r,H1_준내구재_r,H1_total_r,H2_내구재_r,H2_비내구재_r,H2_사치품_r,H2_준내구재_r,H2_total_r,H3_내구재_r,H3_비내구재_r,H3_사치품_r,H3_준내구재_r,H3_total_r
고객번호,,,,,,,,,,,,,,,
1,0.004202,0.693277,0.000000,0.302521,1.0,0.061798,0.716292,0.000000,0.221910,1.0,0.012821,0.750000,0.003205,0.233974,1.0
2,0.032448,0.823009,0.000000,0.144543,1.0,0.011070,0.815498,0.003690,0.169742,1.0,0.019157,0.750958,0.015326,0.214559,1.0
3,0.000000,0.997126,0.000000,0.002874,1.0,0.000000,0.997442,0.000000,0.002558,1.0,0.000000,1.000000,0.000000,0.000000,1.0
4,0.000000,0.851852,0.000000,0.148148,1.0,0.023256,0.761628,0.000000,0.215116,1.0,0.000000,0.908745,0.000000,0.091255,1.0
5,0.002653,0.952255,0.002653,0.042440,1.0,0.015385,0.946154,0.000000,0.038462,1.0,0.000000,0.991870,0.000000,0.008130,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19356,0.000000,1.000000,0.000000,0.000000,1.0,0.002625,0.997375,0.000000,0.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.0
19357,0.000000,0.750000,0.000000,0.250000,1.0,0.000000,0.941176,0.002353,0.056471,1.0,0.002208,0.964680,0.002208,0.030905,1.0
19359,0.000000,1.000000,0.000000,0.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.0


In [24]:
# testlist = ['H1','H2','H3','H4']
# c_li = list(H1_r.columns)

# testList = []
# for col in c_li:
#   for h in testlist:
#     testList.append(f'{h}_{col}')
# test = test[testList]

test = test[['H1_내구재_r',
 'H2_내구재_r',
 'H3_내구재_r',
 'H1_준내구재_r',
 'H2_준내구재_r',
 'H3_준내구재_r',
 'H1_비내구재_r',
 'H2_비내구재_r',
 'H3_비내구재_r',
 'H1_사치품_r',
 'H2_사치품_r',
 'H3_사치품_r'
]]
test = test.reset_index()
test

,고객번호,H1_내구재_r,H2_내구재_r,H3_내구재_r,H1_준내구재_r,H2_준내구재_r,H3_준내구재_r,H1_비내구재_r,H2_비내구재_r,H3_비내구재_r,H1_사치품_r,H2_사치품_r,H3_사치품_r
0,1,0.004202,0.061798,0.012821,0.302521,0.221910,0.233974,0.693277,0.716292,0.750000,0.000000,0.000000,0.003205
1,2,0.032448,0.011070,0.019157,0.144543,0.169742,0.214559,0.823009,0.815498,0.750958,0.000000,0.003690,0.015326
2,3,0.000000,0.000000,0.000000,0.002874,0.002558,0.000000,0.997126,0.997442,1.000000,0.000000,0.000000,0.000000
3,4,0.000000,0.023256,0.000000,0.148148,0.215116,0.091255,0.851852,0.761628,0.908745,0.000000,0.000000,0.000000
4,5,0.002653,0.015385,0.000000,0.042440,0.038462,0.008130,0.952255,0.946154,0.991870,0.002653,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19080,19356,0.000000,0.002625,0.000000,0.000000,0.000000,0.000000,1.000000,0.997375,1.000000,0.000000,0.000000,0.000000
19081,19357,0.000000,0.000000,0.002208,0.250000,0.056471,0.030905,0.750000,0.941176,0.964680,0.000000,0.002353,0.002208
19082,19359,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000
19083,19360,0.000000,0.000000,0.000000,0.000000,0.011070,0.026316,1.000000,0.988930,0.967105,0.000000,0.000000,0.006579


In [25]:
with open('/content/drive/MyDrive/공유문서/1차 프로젝트/5조/1012발표도못하고수정/10월14일이후 새로추가/test/new_ratio.pkl','wb') as test_save:
  pickle.dump(test,test_save)